Here is another installment in our series on HackerRank problem's.  This time, we are looking at the 'medium' rated [Count Triplets](https://www.hackerrank.com/challenges/count-triplets-1/problem?h_l=interview&playlist_slugs%5B%5D=interview-preparation-kit&playlist_slugs%5B%5D=dictionaries-hashmaps).  This one took me some time to see through, and my writeup will be split into multiple articles.

__Problem.__

You are given an array and you need to find number of triplets of indices $(i, j, k)$ with $i<j<k$ such that the elements at those indices are in geometric progression for a given common ratio $r\ge 1$.

__Solution.__

First off:  what exactly is a geometric progression? Well, integers $a_1, a_2$ and $a_3$ are a geometric progression if 

$$(a_1, a_2, a_3) = \left(c\cdot r^i, c\cdot r^{i+1}, c\cdot r^{i+2}\right)$$ 

for some nonnegative integer $i$ and constant $c$.  Here are a few examples:

\begin{align}
2, 4, 8 &= 2^1, 2^2, 2^3 \\
2, 10, 25 &= 2\cdot 5^0, 2\cdot 5^1, 2\cdot 5^2 \\
2, 2, 2 &= 2\cdot 1^0, 2\cdot 1^1, 2\cdot 1^2 \\
\end{align}

Let's start with the imports that we will need.  The `testing` module is my (rudimentary) custom testing suite.  

In [62]:
import shelve
import requests

import itertools as it

from collections import Counter
from functools import reduce

import testing

import matplotlib.pyplot as plt
%matplotlib inline

__Attempt 1__

My first attempt involved two functions:  one to determine whether an array (sorted) was a geometric progression by checking that the ratios of all its successive elements are all the same, and one to count how many such subarrays of length 3 there are in the given array.  The latter function works by running every combination in `itertools.combinations(array)` of length 3 through the first function.  I was not surprised to find that, while it worked for some smaller test cases, the majority failed due to timeouts.  That is a recurring theme in my experience of HackerRank.

In [63]:
def is_geometric_progression(array):
    """Returns if array (sorted) is a geometric progression."""
    return all([array[i+1]/array[i] == array[1]/array[0] for i in range(len(array)-1)])


def count_geo_progs(array, length):
    """Counts number of geometrical of given length in array."""
    count = 0
    for combo in it.combinations(array, length):
        count += is_geometric_progression(combo)
    return count


Before optimizing, however, I noted that one of the small sized test cases was also failing, so some debugging was in order.  

__Testing 1__

In [64]:
# test cases

arrays = [
    [1,4,16,64],
    [1,3,9,9,27,81],
    [1,2,2,4],
    [1,5,5,25,125]
         ]

args = [(array, 3) for array in arrays]

expected = [2, 6, 2, 4]


In [66]:
import testing
testing.test(count_geo_progs, args, expected)

   TESTING count_geo_progs    
Test 0:  count_geo_progs(([1, 4, 16, 64], 3)...)
  Computed: 2
  Expected: 2
------------------------------
Test 1:  count_geo_progs(([1, 3, 9, 9, 27, 81], 3)...)
  Computed: 8
  Expected: 6
8 6
        TEST 1 FAILED         
------------------------------
Test 2:  count_geo_progs(([1, 2, 2, 4], 3)...)
  Computed: 2
  Expected: 2
------------------------------
Test 3:  count_geo_progs(([1, 5, 5, 25, 125], 3)...)
  Computed: 4
  Expected: 4
------------------------------
The following 1 test is failing:
             [1]              


__Debugging 1__

The offending array was `[1, 3, 9, 9, 27, 81]`, but after a bit of thought I realized that the offender was me.  I had not read the instructions carefully enough, and was counting _all_ geometric progressions in the array.  This included two instances of `(1, 9, 81)`, which is a only a progression with $r=9$, and so should no be counted when $r=3$.  This was a straightforward fix.

In [78]:
# debugging 1

def is_geometric_progression(array, r=None):
    """Returns if array (sorted) is a geometric progression."""
    if not r:
        r = array[1]/array[0]
    return all([array[i+1]/array[i] == r for i in range(len(array)-1)])

def count_geo_progs(array, r, length):
    """Counts number of geometrical progressions with common ratio r
    of given length in array."""
    count = 0
    for combo in it.combinations(array, length):
        count += is_geometric_progression(combo, r)
    return count

In [79]:
# updating test cases

arrays = [
    [1,4,16,64],
    [1,3,9,9,27,81],
    [1,2,2,4],
    [1,5,5,25,125]
         ]

ratios = [4, 3, 2, 5]

args = [(array, ratio, 3) for array, ratio in zip(arrays, ratios)]

expected = [2, 6, 2, 4]

testing.test(count_geo_progs, args, expected)

   TESTING count_geo_progs    
Test 0:  count_geo_progs(([1, 4, 16, 64], 4, 3)...)
  Computed: 2
  Expected: 2
------------------------------
Test 1:  count_geo_progs(([1, 3, 9, 9, 27, 81], 3,...)
  Computed: 6
  Expected: 6
------------------------------
Test 2:  count_geo_progs(([1, 2, 2, 4], 2, 3)...)
  Computed: 2
  Expected: 2
------------------------------
Test 3:  count_geo_progs(([1, 5, 5, 25, 125], 5, 3...)
  Computed: 4
  Expected: 4
------------------------------
The following 0 tests are failing:
              []              


So now it is working for the limited amount of small test cases that I am working with here.  But as I said, most tests on HackerRank fail due to timeout.  Tomorrow I will begin to work towards optimizing my functions.